In [2]:
import math
import copy

# Get iris
def get_iris():
    from sklearn import datasets
    iris = datasets.load_iris()
    X = iris.data 
    y = iris.target

    data_iris = []
    for i in range(len(X)):
        dict = {}
        dict['f0'] = X[i][0]
        dict['f1'] = X[i][1]
        dict['f2'] = X[i][2]
        dict['f3'] = X[i][3]

        dict['label'] = y[i]
        data_iris.append(dict)
    return data_iris
    
data = get_iris()
label = 'label'


def entropy(data, label):
    cl = {}
    for x in data:
        if x[label] in cl:
            cl[x[label]] += 1
        else:
            cl[x[label]] = 1
    tot_cnt = sum(cl.values())
    return sum([ -1 * (float(cl[x])/tot_cnt) * math.log2(float(cl[x])/tot_cnt) for x in cl])


def findInformationGain(data, label, column, entropyParent):
    keys = { i[column] for i in data }
    entropies = {}
    count = {}
    avgEntropy = 0
    for val in keys:
        modData = [ x for x in data if x[column] == val]
        entropies[val] = entropy(modData, label)
        count[val] = len(modData)
        avgEntropy += (entropies[val] * count[val])

    tot_cnt = sum(count.values())
    avgEntropy /= tot_cnt
    return entropyParent - avgEntropy

node = 0
nodeMapping = {}
edges = []

def makeDecisionTree(data, label, parent=-1, branch=''):

    global node, nodeMapping
    if parent >= 0:
        edges.append((parent, node, branch))

    #find the variable(column) with maximum information gain
    infoGain = []
    columns = [x for x in data[0]]
    for column in columns:
        if not(column == label):
            ent = entropy(data, label)
            infoGain.append((findInformationGain(data, label, column, ent), column))
    splitColumn = max(infoGain)[1]

    # Leaf node, final result, if maximum information gain is not significant
    if max(infoGain)[0] < 0.01:
        nodeMapping[node] = data[0][label]
        node += 1
        return
    nodeMapping[node] = splitColumn
    parent = node
    node += 1
    branchs = { i[splitColumn] for i in data }#All out-going edges from current node
    for branch in branchs:

        # Create sub table under the current decision branch
        modData = [x for x in data if splitColumn in x and x[splitColumn] == branch]
        for y in modData:
            if splitColumn in y:
                del y[splitColumn]

        # Create sub-tree
        makeDecisionTree(modData, label, parent, branch)

makeDecisionTree(data, label)

print('nodemapping ==> ', nodeMapping, '\n\nedges ===>', edges)


path = []
label_x = None

# QUERY
def query(i, data_x):
    global path, label_x
    path.append(i)
    next_q = False
    for e in edges:
        if e[0]==i:
            next_q=True
            break
        
    if next_q:
        for e in edges:
            if e[0]==i and e[2]==data_x[str(nodeMapping[i])]:
                i = e[1]
                query(i, data_x)
                break
    else:
        label_x = nodeMapping[i]
        
data_x = get_iris()[149]
query(0, data_x)
print()
print('original_data:', data_x)
print('original_path:',path,' predict_label:', label_x)

# ATTACK
attack_label = None
attack_path = None

def judge_e(i):
    next_ = False
    for e in edges:
        if e[0]==i:
            next_=True
            break
    return next_
    
def atk_path(path_,i):
    global attack_label, attack_path
    for e in edges:
        ppath = copy.deepcopy(path_)
        if e[0]==i:
            ppath.append(e[1])
            if judge_e(e[1]):
                atk_path(ppath,e[1])
            elif nodeMapping[e[1]]!=label_x and attack_label==None:
                attack_path = ppath
                attack_label = nodeMapping[e[1]]
            
def attack():
    for i in range(1,len(path)):
        atk_path(path[:-i],path[-1-i])
        if attack_label != None:
            break
            
attack()
print('attack_path:',attack_path,' attack_label:', attack_label)

nodemapping ==>  {0: 'f2', 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 1, 11: 'f3', 12: 2, 13: 1, 14: 1, 15: 1, 16: 'f3', 17: 1, 18: 2, 19: 2, 20: 1, 21: 'f3', 22: 1, 23: 2, 24: 2, 25: 2, 26: 2, 27: 1, 28: 1, 29: 1, 30: 1, 31: 1, 32: 'f3', 33: 1, 34: 2, 35: 2, 36: 2, 37: 2, 38: 2, 39: 2, 40: 2, 41: 2, 42: 2, 43: 2, 44: 2, 45: 2, 46: 2, 47: 2, 48: 1, 49: 1, 50: 1, 51: 1, 52: 1, 53: 'f0', 54: 2, 55: 1, 56: 1, 57: 2, 58: 1, 59: 2, 60: 2, 61: 2, 62: 2, 63: 2, 64: 2, 65: 1} 

edges ===> [(0, 1, 1.7), (0, 2, 1.4), (0, 3, 1.6), (0, 4, 1.3), (0, 5, 1.5), (0, 6, 1.1), (0, 7, 1.2), (0, 8, 1.0), (0, 9, 1.9), (0, 10, 4.7), (0, 11, 4.5), (11, 12, 1.7), (11, 13, 1.5), (11, 14, 1.6), (11, 15, 1.3), (0, 16, 4.9), (16, 17, 1.5), (16, 18, 2.0), (16, 19, 1.8), (0, 20, 4.0), (0, 21, 5.0), (21, 22, 1.7), (21, 23, 2.0), (21, 24, 1.9), (21, 25, 1.5), (0, 26, 6.0), (0, 27, 3.5), (0, 28, 3.0), (0, 29, 4.6), (0, 30, 4.4), (0, 31, 4.1), (0, 32, 5.1), (32, 33, 1.6), (32, 34, 2.0), (32, 35, 2.4), (32